## Imports

In [121]:
# DO NOT CHANGE THESE LINES.
using Suppressor
@suppress begin
    using DataFrames, CSV, Random, Statistics, Serialization, LazyJSON, StatsBase, DecisionTree, ScientificTypes, MLJ
end


## Paths

In [122]:
# DO NOT CHANGE THESE LINES 
ROOT_DIR = dirname(pwd())
MODEL_INPUTS_OUTPUTS = joinpath(ROOT_DIR, "model_inputs_outputs")
INPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = joinpath(INPUT_DIR, "schema")
DATA_DIR = joinpath(INPUT_DIR, "data")
OUTPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "outputs")
TRAIN_DIR = joinpath(DATA_DIR, "training")
TEST_DIR = joinpath(DATA_DIR, "testing")
MODEL_PATH = joinpath(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = joinpath(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = joinpath(MODEL_ARTIFACTS_PATH, "ohe.jld2")
PREDICTOR_DIR_PATH = joinpath(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = joinpath(PREDICTOR_DIR_PATH, "predictor.ser")
IMPUTATION_FILE = joinpath(MODEL_ARTIFACTS_PATH, "imputation.ser")
TOP_CATEGORIES = joinpath(MODEL_ARTIFACTS_PATH, "top_categories.ser")
PREDICTIONS_DIR = joinpath(OUTPUT_DIR, "predictions")
PREDICTIONS_FILE = joinpath(PREDICTIONS_DIR, "predictions.csv")
TARGET_LEVELS = joinpath(MODEL_ARTIFACTS_PATH, "target_levels.ser")



if !isdir(MODEL_ARTIFACTS_PATH)
    mkdir(MODEL_ARTIFACTS_PATH)
end
if !isdir(PREDICTOR_DIR_PATH)
    mkdir(PREDICTOR_DIR_PATH)
end

### Reading the schema

In [123]:
# Reading a schema from a JSON file and extracting features
file_name = first(filter(x -> endswith(x, "json"), readdir(INPUT_SCHEMA_DIR)))
schema_path = joinpath(INPUT_SCHEMA_DIR, file_name)
schema_string = read(schema_path, String)  # Read file content as a string
schema = LazyJSON.parse(schema_string)
features = schema["features"]

# Identifying numeric, categorical, and nullable features
numeric_features = String[]
categorical_features = String[]
nullable_features = String[]

for f in features
    if f["dataType"] == "CATEGORICAL"
        push!(categorical_features, f["name"])
    else
        push!(numeric_features, f["name"])
    end
    if f["nullable"]
        push!(nullable_features, f["name"])
    end
end

# Extracting ID and target features
id_feature = schema["id"]["name"]
target_feature = schema["target"]["name"]
target_classes = schema["target"]["classes"]

4-element LazyJSON.Array{Nothing, String}:
 "acc"
 "good"
 "unacc"
 "vgood"

### Reading test data.

In [124]:
file_name = filter(x -> occursin(".csv", x), readdir(TEST_DIR))[1]
file_path = joinpath(TEST_DIR, file_name)
df = DataFrame(CSV.File(file_path))

,id,buying,maint,doors,persons,lug_boot,safety
,Int64,String7,String7,String7,String7,String7,String7
1,598,high,high,4,2,med,med
2,1674,low,low,4,2,small,low
3,860,high,low,5more,more,med,high
4,432,high,vhigh,2,2,small,low
5,1036,med,high,4,4,small,med
6,793,high,low,3,4,small,med
7,465,high,vhigh,3,2,big,low
8,50,vhigh,vhigh,3,more,med,high
9,131,vhigh,high,2,more,med,high


## Data preprocessing
Note that when we work with testing data, we have to impute using the same values learned during training. This is to avoid data leakage.

In [125]:
imputation_values = open(deserialize, IMPUTATION_FILE)
for column in nullable_features
    df[!, Symbol(column)] .= coalesce.(df[!, Symbol(column)], get(imputation_values, string(column), missing))
end

# Saving the id column in a different variable
ids = df[!, Symbol(id_feature)]

# Dropping the id from the DataFrame
select!(df, Not([Symbol(id_feature)]))

,buying,maint,doors,persons,lug_boot,safety
,String7,String7,String7,String7,String7,String7
1,high,high,4,2,med,med
2,low,low,4,2,small,low
3,high,low,5more,more,med,high
4,high,vhigh,2,2,small,low
5,med,high,4,4,small,med
6,high,low,3,4,small,med
7,high,vhigh,3,2,big,low
8,vhigh,vhigh,3,more,med,high
9,vhigh,high,2,more,med,high


### Encoding
We encode the data using the same encoder that we saved during training.

In [126]:
loaded_top_categories = open(deserialize, TOP_CATEGORIES)

# Function to one-hot encode only the top 3 categories
function one_hot_top_categories!(df, top_categories)
    for (feature, top_cats) in top_categories
        if length(top_cats) == 2  # Handle the binary case
            # Assuming the first category in top_cats is treated as 'true'
            new_col_name = "$(feature)_binary"
            df[!, new_col_name] = df[!, feature] .== top_cats[1]
        else  # Handle the general case
            for cat in top_cats
                new_col_name = "$(feature)_$(cat)"
                df[!, new_col_name] = df[!, feature] .== cat
            end
        end
        select!(df, Not(Symbol(feature)))  # Drop the original feature column
    end
end



one_hot_top_categories!(df, loaded_top_categories)

### Making predictions & Creating Predictions DataFrame
Using the model saved during training.

In [131]:
function convert_to_string(item)
    if typeof(item) <: Number
        return string(item)
    else
        # Strip surrounding quotes if the item is a string
        return strip(String(item), '"')
    end
end

model = open(deserialize, PREDICTOR_FILE_PATH)
predictions = apply_forest_proba(model, Matrix(df), [convert_to_string(item) for item in target_classes])
predictions = DataFrame(predictions, Symbol.(target_classes))
insertcols!(predictions, 1, id_feature => ids)
CSV.write(PREDICTIONS_FILE, predictions)

"/Users/moo/Desktop/Upwork/rt-ML/Regression/Julia/Julia-Random-Forest-Classifier-Template/model_inputs_outputs/outputs/predictions/predictions.csv"